## VIDA Seminar Experiments Viz

### Imports

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from imports import *

In [27]:
from models import *
from data import * 
from env import *
from harmonize import *
from sim import *
from sim import plot

In [32]:
# overall imports
import importlib
import data

# sim class
# importlib.reload(sim)

import sim.sim 
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
#importlib.reload(sim.sim_run)
from sim.sim_run import single_sim_run, open_pickled_results
importlib.reload(sim.sim)
importlib.reload(sim.sim_utils)

import sim.plot
from sim.plot import (
    plot_single_model_predictions_with_metrics,
    plot_fold_performance,
    plot_summary_measure_comparison, 
    compare_simulation_results,
    get_sim_performance, 
    get_aggregate_performance
)
importlib.reload(sim.plot)


<module 'sim.plot' from '/scratch/asr655/neuroinformatics/GeneEx2Conn/sim/plot.py'>

#### Check job specs

In [33]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

GPUtil.getGPUs()
print(f"Number of available GPUs: {torch.cuda.device_count()}")

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

GPUtil.showUtilization()

torch.cuda.empty_cache()


CPU Usage: 31.0%
RAM Usage: 15.5%
Available RAM: 318.5G
Total RAM: 377.1G
52.4G
Number of available GPUs: 0


RuntimeError: Could not find an available GPU after 1 attempts with 900 seconds interval.

### Plots to create: 
- PLS, Ridge, XGBoost, MLP performance for structure with spectral embeddings and transcriptome
(will have to add in bilinear later) for community splits 
- PLS, Ridge, XGBoost, MLP performance for combined structure with spectral embeddings and transcriptome for community splits

In [29]:
feature_types = ['transcriptome', 'structural_spectralA_10']
model_types = ['pls', 'ridge', 'xgboost', 'mlp']

local_sim_path = os.getcwd() + '/sim/sim_results/'
SFN_sim_path = os.path.join(os.path.dirname(os.getcwd()), 'GeneEx2Conn_backup/sim/sim_results/SFN_runs_mse/')
print(local_sim_path)
print(SFN_sim_path)

/scratch/asr655/neuroinformatics/GeneEx2Conn/sim/sim_results/
/scratch/asr655/neuroinformatics/GeneEx2Conn_backup/sim/sim_results/SFN_runs_mse/


#### Model test performance 



In [30]:
os.listdir(local_sim_path)


['structural_spectralA_-20_SC_xgboost_random_42_bayes_mse_search.pickle',
 'structural_spectralL_-60_FC_xgboost_random_42_bayes_mse_search.pickle',
 'structural_spectralA_10_FC_mlp_community101_42_grid_mse_search.pickle',
 'transcriptome_FC_xgboost_random_2_bayes_mse_search.pickle',
 'structural_spectralL_-60_SC_xgboost_random_42_bayes_mse_search.pickle',
 'transcriptome_FC_mlp_community101_5_grid_mse_search.pickle',
 'structural_spectralL_10_FC_ridge_random_42_grid_mse_search.pickle',
 'structural_spectralL_3_FC_xgboost_random_42_bayes_mse_search.pickle',
 'transcriptome_SC_xgboost_community101_42_bayes_mse_search.pickle',
 'transcriptome structural_spectralA_10_FC_mlp_community101_42_grid_mse_search.pickle',
 'structural_spectralL_20_FC_ridge_random_42_grid_mse_search.pickle',
 'transcriptome_mlp_community101_1_grid_pearsonsearch.pickle',
 'transcriptome_mlp_random_42_grid_pearsonsearch.pickle',
 'structural euclidean_SC_xgboost_random_2_bayes_pearson_search.pickle',
 'structural_spe

In [79]:
# Get performance stats for a single simulation
stats = get_sim_performance(
    feature_type='transcriptome structural_spectralA_10',
    cv_type='community',
    model_type='mlp',
    #summary_measure='10',
    resolution=1.01,
    random_seed=5,
    metric='pearson_corr'
)

if stats:
    print(f"Train Performance: {stats['train_mean']:.3f} ± {stats['train_stderr']:.3f}")
    print(f"Test Performance: {stats['test_mean']:.3f} ± {stats['test_stderr']:.3f}")

Train Performance: 0.578 ± 0.043
Test Performance: 0.616 ± 0.025


STILL TO RUN: 
- xgboost for structural spectralA [done]
- ridge, pls, and xgboost for combined [running]
- ridge, pls, xgboost for transcriptome alone [running]

In [ ]:
# Get aggregate performance across multiple resolutions and seeds
stats = get_aggregate_performance(
    feature_type='structural_spectralA',
    cv_type='community',
    model_type='mlp',
    resolutions=[1.01],
    random_seeds=[1, 2, 4, 5, 42],
    summary_measure='10',
    metric='pearson_corr'
)

if stats:
    print(f"\nAggregate Performance:")
    print(f"Train: {stats['train_mean']:.3f} ± {stats['train_stderr']:.3f}")
    print(f"Test:  {stats['test_mean']:.3f} ± {stats['test_stderr']:.3f}")
    print(f"Based on {stats['n_runs']} successful runs")

Genetics MLP runs for community splits

In [ ]:
resolutions = [1.01]
#seeds = [1, 2, 4, 5, 42]
seeds = [42]

for r in resolutions:
    for s in seeds:
        print('resolution', r)
        print('seed', s)
        single_sim_run(
            cv_type='community',
            random_seed=s,
            resolution=r,
            model_type='mlp',
            feature_type=['transcriptome'],
            use_gpu=True,
            use_shared_regions=False,
            test_shared_regions=False,
            save_sim=True,
            search_method=('grid', 'mse')
            )

SpectralA MLP runs for community splits

In [ ]:
resolutions = [1.01]
seeds = [1, 2, 4, 5, 42]

for r in resolutions:
    for s in seeds:
        print('resolution', r)
        print('seed', s)
        single_sim_run(
            cv_type='community',
            random_seed=s,
            resolution=r,
            model_type='mlp',
            feature_type=['structural_spectralA'],
            summary_measure='10', # 10 or 20 are the candidates here (start with 10)
            use_gpu=True,
            use_shared_regions=False,
            test_shared_regions=False,
            save_sim=True,
            search_method=('grid', 'mse')
            )

resolution 1.01
seed 1
computing eig of laplacian
computing eig of adjacency
Number of components explaining 95.0% of the variance: 34

 Test fold num: 1
(6320, 20) (6320,) (1122, 20) (1122,)
SEARCH METHOD ('grid', 'mse')
2
3
GPU model input size 20
Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch [10/200], Loss: 0.0161
Epoch [20/200], Loss: 0.0129
Epoch [30/200], Loss: 0.0121
Epoch [40/200], Loss: 0.0102
Epoch [50/200], Loss: 0.0087
Epoch [60/200], Loss: 0.0081
Epoch [70/200], Loss: 0.0074
Epoch [80/200], Loss: 0.0071
Epoch [90/200], Loss: 0.0064
Epoch [100/200], Loss: 0.0061
Epoch [110/200], Loss: 0.0066
Epoch [120/200], Loss: 0.0063
Epoch [130/200], Loss: 0.0058
Epoch [140/200], Loss: 0.0060
Epoch [150/200], Loss: 0.0054
Epoch [160/200], Loss: 0.0055
Epoch [170/200], Loss: 0.0061
Epoch [180/200], Loss: 0.0057
Epoch [190/200], Loss: 0.0050
Epoch [200/200], Loss: 0.0050
[CV 1/2] END batch_size=32, epochs=200, l2_reg=0.001, lr=0.001;, score=(train=-204.837, test=-82.730

Genetics + SpectralA MLP runs for community splits

In [ ]:
resolutions = [1.01]
seeds = [1, 2, 4, 5, 42]

for r in resolutions:
    for s in seeds:
        print('resolution', r)
        print('seed', s)
        single_sim_run(
            cv_type='community',
            random_seed=s,
            resolution=r,
            model_type='mlp',
            feature_type=['transcriptome', 'structural_spectralA'],
            summary_measure='10', # 10 or 20 are the candidates here (start with 10)
            use_gpu=True,
            use_shared_regions=False,
            test_shared_regions=False,
            save_sim=True,
            search_method=('grid', 'mse')
            )

SpectralA linear model runs for community splits


In [6]:
# RIDGE

resolutions = [1.01]
seeds = [1, 2, 4, 5, 42]

for r in resolutions:
    for s in seeds:
        print('resolution', r)
        print('seed', s)
        single_sim_run(
            cv_type='community',
            random_seed=s,
            resolution=r,
            model_type='ridge',
            feature_type=['structural_spectralA'],
            summary_measure='10', # 10 or 20 are the candidates here (start with 10)
            use_gpu=False,
            use_shared_regions=False,
            test_shared_regions=False,
            save_sim=True,
            search_method=('grid', 'mse')
            )

resolution 1.01
seed 1
computing eig of laplacian
computing eig of adjacency
Number of components explaining 95.0% of the variance: 34

 Test fold num: 1
(6320, 20) (6320,) (1122, 20) (1122,)
SEARCH METHOD ('grid', 'mse')
2
3
Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END .............alpha=0, solver=auto;, score=-0.078 total time=   0.0s
[CV 2/2] END .............alpha=0, solver=auto;, score=-0.063 total time=   0.0s
[CV 1/2] END .........alpha=0.001, solver=auto;, score=-0.078 total time=   0.0s
[CV 2/2] END .........alpha=0.001, solver=auto;, score=-0.063 total time=   0.0s
[CV 1/2] END ..........alpha=0.01, solver=auto;, score=-0.078 total time=   0.0s
[CV 2/2] END ..........alpha=0.01, solver=auto;, score=-0.063 total time=   0.0s
[CV 1/2] END ...........alpha=0.1, solver=auto;, score=-0.077 total time=   0.0s
[CV 2/2] END ...........alpha=0.1, solver=auto;, score=-0.064 total time=   0.0s
[CV 1/2] END ...........alpha=1.0, solver=auto;, score=-0.070 tota

In [ ]:
# PLS

resolutions = [1.01]
seeds = [1, 2, 4, 5, 42]

for r in resolutions:
    for s in seeds:
        print('resolution', r)
        print('seed', s)
        single_sim_run(
            cv_type='community',
            random_seed=s,
            resolution=r,
            model_type='pls',
            feature_type=['structural_spectralA'],
            summary_measure='10', # 10 or 20 are the candidates here (start with 10)
            use_gpu=False,
            use_shared_regions=False,
            test_shared_regions=False,
            save_sim=True,
            search_method=('grid', 'mse')
            )